In [2]:
import re
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tf2onnx
import onnx
import onnxruntime as ort

c:\Users\hunte\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv('twitter_parsed_dataset.csv', usecols=[2,4])

In [4]:
stopwords = ['rt','http', 'https:', 'http:']

In [5]:
df = df.rename(columns={'oh_label': 'Label'})
for index in df.index:
    try:
        df['Text'][index]=re.sub('@[^\s]+','',df['Text'][index])
    except:
        pass
df.replace(to_replace="[^a-zA-Z]", value=" ", regex=True, inplace=True)
df['Text'] = df['Text'].str.lower()
for i in stopwords:
    df['Text'] = df['Text'].str.replace(i, '')
df.dropna()

C:\Users\hunte\AppData\Local\Temp/ipykernel_20096/2922597320.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'][index]=re.sub('@[^\s]+','',df['Text'][index])


,Text,Label
0,i read them in context no change in meaning...,0.0
1,now you idiots claim that people who tried to...,0.0
2,call me sexist but when i go to an auto pla...,1.0
3,wrong isis follows the example of mohammed a...,1.0
4,mkr no no no no no no,0.0
...,...,...
16846,feeling so sorry for the girls they should be...,0.0
16847,mkr pretty good dishes we re happy with o...,0.0
16848,deconstructed lemon ta can we please go ...,0.0
16849,you are too stupid to talk to blocked,0.0


In [6]:
df=df.dropna(subset=['Label'])

In [7]:
train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index).sample(frac=1.0) 

In [8]:
train_data = train['Text'].astype('str')
train_label = train['Label']
test_data = test['Text'].astype('str')
test_label = test['Label']

In [9]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(train_data)
tokenizer.fit_on_texts(test_data)

In [10]:
maxlen=50
def get_sequences(tokenizer, tokened):
    sequences = tokenizer.texts_to_sequences(tokened)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

In [11]:
train_data_padded = get_sequences(tokenizer, train_data)
test_data_padded = get_sequences(tokenizer, test_data)

In [13]:
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,16,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(6, activation='softmax')
])

In [14]:
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [15]:
history = model.fit(
    train_data_padded,
    train_label,
    epochs=1, )

422/422 [==============================] - 20s 32ms/step - loss: 0.5848 - accuracy: 0.7486


In [16]:
loss, accuracy = model.evaluate(test_data_padded, test_label)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

106/106 [==============================] - 3s 10ms/step - loss: 0.4092 - accuracy: 0.8220
Loss:  0.409199595451355
Accuracy:  0.8219584822654724


In [17]:
input_signature = [tf.TensorSpec([None, None], tf.int32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)

In [18]:
onnx.save(onnx_model, "charlie.onnx")

In [19]:
onnx_model = onnx.load("charlie.onnx")
onnx.checker.check_model(onnx_model)

In [20]:
ort_sess = ort.InferenceSession('model.onnx')
outputs = ort_sess.run(None, {'x': test_data_padded})


In [21]:
outputs[0]

array([[9.39577401e-01, 5.99975474e-02, 1.30807282e-04, 5.77332939e-05,
        6.79429359e-05, 1.68567727e-04],
       [9.32920158e-01, 6.66228384e-02, 1.45630649e-04, 6.37666744e-05,
        7.32636618e-05, 1.74334797e-04],
       [8.79249454e-01, 1.20100670e-01, 2.54144717e-04, 1.07261643e-04,
        1.05611092e-04, 1.82828866e-04],
       ...,
       [8.45851243e-01, 1.53399035e-01, 3.12716118e-04, 1.30782035e-04,
        1.20025965e-04, 1.86190300e-04],
       [9.42601487e-02, 9.03148949e-01, 1.04690541e-03, 6.77421398e-04,
        4.19969758e-04, 4.46591468e-04],
       [9.35477793e-01, 6.40775710e-02, 1.40489428e-04, 6.12457734e-05,
        7.13842237e-05, 1.71494845e-04]], dtype=float32)